In [12]:
import torch 
import visdom
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# prepare the visdom envirement
viz = visdom.Visdom(env='beamforming', server='10.15.89.41', port=38720, use_incoming_socket=False)

# 1. Prepare Dataset
G = 3
N = 50
K = 5
batchSize = 200


class BeamformingDataset(Dataset):
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter = ',', dtype = np.float32)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, :-2*G*K])
        self.y_data = torch.from_numpy(xy[:, 2*N*G*K:])
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

## Load the trainingSet and testingSet
trainingSet = BeamformingDataset('trainingData.csv.gz')
testingSet = BeamformingDataset('testingData.csv.gz')
train_loader = DataLoader(dataset = trainingSet, batch_size=batchSize, shuffle=True, num_workers=8)
test_loader = DataLoader(dataset = testingSet, batch_size=batchSize, shuffle=False, num_workers=8)

# 2. Design Model
class BeamformingNet(torch.nn.Module):
    def __init__(self):
        super(BeamformingNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 8, kernel_size = 3, padding = 1)
        # self.conv1 = torch.nn.DataParallel(self.conv1)
        self.conv2 = torch.nn.Conv2d(8, 8, kernel_size = 3, padding = 1)
        # self.conv2 = torch.nn.DataParallel(self.conv2)
        
        self.norm = torch.nn.BatchNorm2d(8, eps = 1e-03, momentum = 0.99)
    
        self.fc = torch.nn.Linear(12000, 2*G*K)
        
    def forward(self, x):
        BATCH_SIZE = x.size(0)
        
        x = x.view(BATCH_SIZE, 1, 1, -1)
        x = F.relu(self.norm(self.conv1(x)))
        x = F.relu(self.norm(self.conv2(x)))
        x = x.view(BATCH_SIZE, -1) #flatten
        x = self.fc(x)
        return x
        x = F.Sigmoid(x)
    
model = BeamformingNet()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# 3. Construct Loss and Optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

# 4. Train and Test
def train(epoch):
    running_loss = 0.0
    train_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_loss += loss.item()
        if batch_idx % 2 == 1:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 2))
            running_loss = 0.0
    viz.line([train_loss], [epoch], win='train_loss', update='append')
            
def test():
    with torch.no_grad():
        running_loss = 0.0
        for data in test_loader:
            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            
            # compute the loss
            outputs = model(inputs)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            
        print('Running loss on test set: %.3f' % running_loss)
        
viz.line([0.], [0], win='train_loss', opts=dict(title='train_loss'))
for epoch in range(20):
    train(epoch)
    test()
            

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.
failed CONNECT via proxy status: 403
failed CONNECT via proxy status: 403
failed CONNECT via proxy status: 403
failed CONNECT via proxy status: 403
failed CONNECT via proxy status: 403
failed CONNECT via proxy status: 403
failed CONNECT via proxy status: 403
failed CONNECT via proxy status: 403


[1,     2] loss: 1508.363
[1,     4] loss: 1471.709
[1,     6] loss: 1494.134
[1,     8] loss: 1489.619
[1,    10] loss: 1512.054
[1,    12] loss: 1491.034
[1,    14] loss: 1480.291
[1,    16] loss: 1484.556
[1,    18] loss: 1488.584
[1,    20] loss: 1490.449
[1,    22] loss: 1482.696
[1,    24] loss: 1459.649
[1,    26] loss: 1442.905
[1,    28] loss: 1460.367
[1,    30] loss: 1444.336
[1,    32] loss: 1416.643
[1,    34] loss: 1421.786
[1,    36] loss: 1411.623
[1,    38] loss: 1388.725
[1,    40] loss: 1420.643
[1,    42] loss: 1384.911
[1,    44] loss: 1359.018
[1,    46] loss: 1366.466
[1,    48] loss: 1373.813
[1,    50] loss: 1358.705
[1,    52] loss: 1343.786
[1,    54] loss: 1328.463
[1,    56] loss: 1340.003
[1,    58] loss: 1322.870
[1,    60] loss: 1356.004
[1,    62] loss: 1332.193
[1,    64] loss: 1313.434
[1,    66] loss: 1299.124
[1,    68] loss: 1305.334
[1,    70] loss: 1292.227
[1,    72] loss: 1269.012
[1,    74] loss: 1280.259
[1,    76] loss: 1266.027
[1,    78] l

failed CONNECT via proxy status: 403


Running loss on test set: 29116.391
[2,     2] loss: 1124.289
[2,     4] loss: 1133.138
[2,     6] loss: 1151.255
[2,     8] loss: 1101.430
[2,    10] loss: 1073.820
[2,    12] loss: 1084.060
[2,    14] loss: 1078.298
[2,    16] loss: 1097.016
[2,    18] loss: 1025.069
[2,    20] loss: 1060.799
[2,    22] loss: 1046.692
[2,    24] loss: 1027.150
[2,    26] loss: 1054.553
[2,    28] loss: 999.875
[2,    30] loss: 1011.008
[2,    32] loss: 1027.352
[2,    34] loss: 1013.125
[2,    36] loss: 998.562
[2,    38] loss: 989.248
[2,    40] loss: 956.610
[2,    42] loss: 970.326
[2,    44] loss: 975.104
[2,    46] loss: 936.360
[2,    48] loss: 922.861
[2,    50] loss: 933.911
[2,    52] loss: 900.303
[2,    54] loss: 919.651
[2,    56] loss: 893.046
[2,    58] loss: 895.911
[2,    60] loss: 902.159
[2,    62] loss: 883.581
[2,    64] loss: 872.235
[2,    66] loss: 874.100
[2,    68] loss: 860.467
[2,    70] loss: 849.048
[2,    72] loss: 822.199
[2,    74] loss: 843.775
[2,    76] loss: 843.19

failed CONNECT via proxy status: 403


...

[16,    88] loss: 0.964
[16,    90] loss: 0.872
[16,    92] loss: 0.891
[16,    94] loss: 0.961
[16,    96] loss: 0.899
[16,    98] loss: 1.034
[16,   100] loss: 1.185
Running loss on test set: 42.657
[17,     2] loss: 0.964
[17,     4] loss: 0.856
[17,     6] loss: 0.746
[17,     8] loss: 0.814
[17,    10] loss: 0.803
[17,    12] loss: 0.790
[17,    14] loss: 0.747
[17,    16] loss: 0.724
[17,    18] loss: 0.743
[17,    20] loss: 0.751
[17,    22] loss: 0.756
[17,    24] loss: 0.826
[17,    26] loss: 0.801
[17,    28] loss: 0.768
[17,    30] loss: 0.764
[17,    32] loss: 0.863
[17,    34] loss: 0.779
[17,    36] loss: 0.804
[17,    38] loss: 0.815
[17,    40] loss: 0.762
[17,    42] loss: 0.782
[17,    44] loss: 0.859
[17,    46] loss: 0.908
[17,    48] loss: 0.815
[17,    50] loss: 0.847
[17,    52] loss: 0.906
[17,    54] loss: 0.794
[17,    56] loss: 0.832
[17,    58] loss: 0.868
[17,    60] loss: 0.848
[17,    62] loss: 0.763
[17,    64] loss: 0.856
[17,    66] loss: 0.911
[17,   

failed CONNECT via proxy status: 403


[18,     2] loss: 0.881
[18,     4] loss: 0.739
[18,     6] loss: 0.692
[18,     8] loss: 0.816
[18,    10] loss: 0.800
[18,    12] loss: 0.680
[18,    14] loss: 0.648
[18,    16] loss: 0.685
[18,    18] loss: 0.706
[18,    20] loss: 0.669
[18,    22] loss: 0.645
[18,    24] loss: 0.703
[18,    26] loss: 0.764
[18,    28] loss: 0.757
[18,    30] loss: 0.743
[18,    32] loss: 0.728
[18,    34] loss: 0.801
[18,    36] loss: 0.912
[18,    38] loss: 0.782
[18,    40] loss: 0.766
[18,    42] loss: 0.727
[18,    44] loss: 0.841
[18,    46] loss: 0.842
[18,    48] loss: 0.733
[18,    50] loss: 0.696
[18,    52] loss: 0.695
[18,    54] loss: 0.672
[18,    56] loss: 0.731
[18,    58] loss: 0.766
[18,    60] loss: 0.912
[18,    62] loss: 0.903
[18,    64] loss: 0.952
[18,    66] loss: 0.875
[18,    68] loss: 0.899
[18,    70] loss: 0.797
[18,    72] loss: 0.691
[18,    74] loss: 0.803
[18,    76] loss: 0.781
[18,    78] loss: 0.689
[18,    80] loss: 0.695
[18,    82] loss: 0.716
[18,    84] loss

failed CONNECT via proxy status: 403


Running loss on test set: 25.071
[20,     2] loss: 0.564
[20,     4] loss: 0.621
[20,     6] loss: 0.803
[20,     8] loss: 0.775
[20,    10] loss: 0.683
[20,    12] loss: 0.568
[20,    14] loss: 0.762
[20,    16] loss: 0.800
[20,    18] loss: 0.737
[20,    20] loss: 0.711
[20,    22] loss: 0.686
[20,    24] loss: 0.621
[20,    26] loss: 0.558
[20,    28] loss: 0.547
[20,    30] loss: 0.542
[20,    32] loss: 0.586
[20,    34] loss: 0.635
[20,    36] loss: 0.594
[20,    38] loss: 0.586
[20,    40] loss: 0.642
[20,    42] loss: 0.572
[20,    44] loss: 0.552
[20,    46] loss: 0.538
[20,    48] loss: 0.573
[20,    50] loss: 0.594
[20,    52] loss: 0.612
[20,    54] loss: 0.579
[20,    56] loss: 0.528
[20,    58] loss: 0.533
[20,    60] loss: 0.536
[20,    62] loss: 0.569
[20,    64] loss: 0.585
[20,    66] loss: 0.574
[20,    68] loss: 0.548
[20,    70] loss: 0.522
[20,    72] loss: 0.533
[20,    74] loss: 0.564
[20,    76] loss: 0.572
[20,    78] loss: 0.618
[20,    80] loss: 0.625
[20,   

In [13]:
# Generate the dataset
import numpy as np

np.random.seed(0)

G = 3
N = 50
K = 5
trainingSample = 20000
testingSample = 5000

trainingInputs = np.random.randn(trainingSample, 2*N*G*K)
testingInputs = np.random.randn(testingSample, 2*N*G*K)

# outputs size: 2*G*k
a = np.random.randn(2*N*G*K, 2*G*K)
trainingOutputs = np.dot(trainingInputs, a)
testingOutputs = np.dot(testingInputs, a)

trainingData = np.hstack((trainingInputs, trainingOutputs))
testingData = np.hstack((testingInputs, testingOutputs))

np.savetxt('trainingData.csv.gz', trainingData, fmt='%5f', delimiter=',')
np.savetxt('testingData.csv.gz', testingData, fmt='%5f', delimiter=',')

In [8]:
import torch 
import visdom
import numpy as np

vis = visdom.Visdom(env='test', server='10.15.89.41', port=38720, use_incoming_socket=False)

##
trace = dict(x=[1, 2, 3], y=[4, 5, 6], mode="markers+lines", type='custom',
             marker={'color': 'red', 'symbol': 104, 'size': "10"},
             text=["one", "two", "three"], name='1st Trace')
layout = dict(title="First Plot", xaxis={'title': 'x'}, yaxis={'title': 'y'})

vis._send({'data': [trace], 'layout': layout, 'win': 'mywin'})

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


'mywin'